Import necessary packages

In [ ]:
import os
import json
import numpy
import datetime
import pandas as pd
from sqlalchemy import create_engine

Read in file

In [ ]:
file = r'/Users/tilliedavies/Downloads/Trees.csv'
df = pd.read_csv (file)

Create SQL Connection

In [ ]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "donotforget!5"
src_dbname = "trees_dw"
dst_dbname = "trees_dw"

In [ ]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

Remove unneccesary columns

In [ ]:
df.drop (['CONDITION', 'CONDITION_ASSESSMENT_DATE', 'CURRENT_STATUS_DATE', 'ASBUILTPLANNO'], axis=1, inplace=True)
df.columns

Insert CSV File into SQL Database

In [ ]:
db_operation = "insert"
set_dataframe(user_id, pwd, host_name, dst_dbname, df, 'trees_table', 'OBJECTID', db_operation)

Generate a brief summary of the data file ingestion

In [ ]:
records = len(df)
print("records =", records)
columns = len(list(df.columns))
print("columns =", columns)

Try / Catch Errors

In [ ]:
def get_csv_response(df, headers, params):
    try:
        response = requests.get(df, headers=headers, params=params)
        response.raise_for_status()
    
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the CSV occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.InvalidHeader as erri:
        return "A Header Error occurred: " + repr(erri)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)
        
    return response.csv()